In [2]:
from transformers import pipeline

# Hugging Face pipeline으로 질문 분석
question = "탑 아트록스의 아이템은 뭐야?"
qa_pipeline = pipeline("feature-extraction")  # 또는 custom tokenizer 사용
keywords = ["아이템"]  # 우선적으로 찾을 키워드 리스트

if any(keyword in question for keyword in keywords):
    print("아이템 관련 API를 호출합니다.")

No model was supplied, defaulted to distilbert/distilbert-base-cased and revision 6ea8117 (https://huggingface.co/distilbert/distilbert-base-cased).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


아이템 관련 API를 호출합니다.


In [3]:
import pandas as pd

def extract_champion_names(csv_file_path):
    df = pd.read_csv(csv_file_path)  # CSV 읽기
    return df["name"].tolist()  # 'name' 열을 리스트로 변환

# CSV 파일 경로
file_path = "./Champion_list.csv"
champion_names = extract_champion_names(file_path)
print(champion_names)
# 출력: ['아트록스', '아리', '이즈리얼', '쓰레쉬', .....]
# 리그오브레전드의 모든 챔피언들의 이름을 불러온다..


['아트록스', '아리', '아칼리', '아크샨', '알리스타', '아무무', '애니비아', '애니', '아펠리오스', '애쉬', '아우렐리온 솔', '아지르', '바드', '벨베스', '블리츠크랭크', '브랜드', '브라움', '브라이어', '케이틀린', '카밀', '카시오페아', '초가스', '코르키', '다리우스', '다이애나', '드레이븐', '문도 박사', '에코', '엘리스', '이블린', '이즈리얼', '피들스틱', '피오라', '피즈', '갈리오', '갱플랭크', '가렌', '나르', '그라가스', '그레이브즈', '그웬', '헤카림', '하이머딩거', '흐웨이', '일라오이', '이렐리아', '아이번', '잔나', '자르반 4세', '잭스', '제이스', '진', '징크스', '카이사', '칼리스타', '카르마', '카서스', '카사딘', '카타리나', '케일', '케인', '케넨', '카직스', '킨드레드', '클레드', '코그모', '크산테', '르블랑', '리 신', '레오나', '릴리아', '리산드라', '루시안', '룰루', '럭스', '말파이트', '말자하', '마오카이', '마스터 이', '밀리오', '미스 포츈', '오공', '모데카이저', '모르가나', '나피리', '나미', '나서스', '노틸러스', '니코', '니달리', '닐라', '녹턴', '누누와 윌럼프', '올라프', '오리아나', '오른', '판테온', '뽀삐', '파이크', '키아나', '퀸', '라칸', '람머스', '렉사이', '렐', '레나타 글라스크', '레넥톤', '렝가', '리븐', '럼블', '라이즈', '사미라', '세주아니', '세나', '세라핀', '세트', '샤코', '쉔', '쉬바나', '신지드', '사이온', '시비르', '스카너', '스몰더', '소나', '소라카', '스웨인', '사일러스', '신드라', '탐 켄치', '탈리야', '탈론', '타릭', '티모', '쓰레쉬', '트리스타나', '트런들', '트린다미어', '트위스티드 페이트'

In [4]:
from konlpy.tag import Okt


roles = ["탑", "정글", "미드", "원딜", "서폿"]

# 형태소 분석기 객체 생성
okt = Okt()

# 역할과 챔피언 이름 추출 함수
def extract_entities(sentence):
    # 형태소 분석
    morphs = okt.morphs(sentence)

    # 역할과 챔피언 이름 초기화
    extracted_role = None
    extracted_champion = None

    # 형태소 중에서 역할과 챔피언 이름을 찾음
    for word in morphs:
        if word in roles:
            extracted_role = word
        if word in champion_names:
            extracted_champion = word

    return {"role": extracted_role, "champion": extracted_champion}

# 테스트 문장
question = "미드 아리 아이템 알려줘"
entities = extract_entities(question)
print(entities)
# 출력 예시: {'role': '탑', 'champion': '아트록스'}


{'role': '미드', 'champion': '아리'}


In [5]:
def format_item_info(item_data, role, champion):
    if 'error' in item_data:
        return item_data['error']
    
    # 역할과 챔피언을 포함한 동적 메시지 생성
    response = f"챔피언 {champion} ({role})에 대한 추천 아이템과 룬을 안내해 드리겠습니다.\n"

    # 주 룬 포맷팅
    primary_runes = "주 룬: " + ", ".join([f"{rune['name']} ({rune['category']})" for rune in item_data['labeledPrimaryRunes']])
    response += primary_runes + "\n"
    
    # 부 룬 포맷팅
    sub_runes = "부 룬: " + ", ".join([f"{rune['name']} ({rune['category']})" for rune in item_data['labeledSubRunes']])
    response += sub_runes + "\n"
    
    # 추천 아이템 포맷팅
    items = "추천 아이템: " + ", ".join([item['name'] for item in item_data['labeledItems']])
    response += items

    return response

def format_user_stats(stats_data):
    # stats_data가 리스트 형태일 경우 처리
    if isinstance(stats_data, list):
        # 각 챔피언의 전적 정보를 포맷팅하여 출력
        response = "전적 정보에 대해 알려 드리겠습니다!!\n"
        for stat in stats_data:
            if 'championName' in stat:
                response += f"\n챔피언: {stat.get('championName', '정보 없음')}\n"
                response += f"킬: {stat.get('kills', '정보 없음')}\n"
                response += f"데스: {stat.get('deaths', '정보 없음')}\n"
                response += f"어시스트: {stat.get('assists', '정보 없음')}\n"
                response += f"딜량: {stat.get('dealt', '정보 없음')}\n"
                response += f"피해량: {stat.get('taken', '정보 없음')}\n"
                response += f"딜량/분: {stat.get('dealt_per_minute', '정보 없음')}\n"
                response += f"피해량/분: {stat.get('taken_per_minute', '정보 없음')}\n"
                response += f"딜량/피해량 비율: {stat.get('dealt_per_taken', '정보 없음')}\n"
                response += f"시야: {stat.get('vision', '정보 없음')}\n"
                response += f"힐: {stat.get('heal', '정보 없음')}\n"
                response += f"CS: {stat.get('cs', '정보 없음')}\n"
                response += f"게임 시간(초): {stat.get('duration', '정보 없음')}\n"
            else:
                response += "잘못된 데이터 형식입니다."
        return response
    else:
        return "전적 데이터 형식이 잘못되었습니다."




In [6]:
import requests
from konlpy.tag import Okt
from retrying import retry
import re

okt = Okt()

item_keywords = ["아이템"]
stat_keywords = ["전적", "승률"]

# API 예시 URL들 (실제 API URL로 교체 필요)
stat_api_url = "http://localhost:3000/summonerInfo"
item_api_url = "http://localhost:3000/getChampInfo"

winrate_api_url = "https://api.example.com/winrate" #승률예측 부분 api는 서버에 올리기 번거로움 tensorflow를 챗봇에게 내장시킬 예정

def get_item_info(position, champion):
    url = f"{item_api_url}/{position}/{champion}"
    try:
        response = requests.get(url, timeout=20)  # 10초 동안 기다림
        if response.status_code == 200:
            return response.json()  # 아이템 리스트 반환
        return {"error": f"Failed to fetch item info: {response.status_code}"}
    except requests.exceptions.Timeout:
        return {"error": "Request timed out"}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def get_user_stats(nickname, tag):
    url = f"{stat_api_url}/{nickname}/{tag}"
    try:
        # 요청에 10초 타임아웃 설정
        response = requests.get(url, timeout=20)
        if response.status_code == 200:
            return response.json()  # 유저 전적 정보 반환
        return {"error": f"Failed to fetch user stats: {response.status_code}"}
    except requests.exceptions.Timeout:
        return {"error": "Request timed out"}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}  # 기타 네트워크 에러 처리

def get_winrate(champion):
    response = requests.get(winrate_api_url, params={"champion": champion})
    if response.status_code == 200:
        return response.json()  # 승률 정보 반환
    return {}

# 역할, 챔피언, 아이템, 닉네임, 태그 추출 함수
def extract_entities(sentence):
    morphs = okt.morphs(sentence)

    extracted_role = None
    extracted_champion = None
    extracted_item = None
    extracted_stat = None
    extracted_nickname = None
    extracted_tag = None

    # 닉네임#태그 형식 추출 (정규 표현식 사용)
    match = re.search(r"(\S+)#(\S+)", sentence)
    if match:
        extracted_nickname = match.group(1)  # 닉네임
        extracted_tag = match.group(2)       # 태그

    # 형태소 중에서 역할과 챔피언 이름을 찾음
    for word in morphs:
        if word in roles:
            extracted_role = word
        if word in champion_names:
            extracted_champion = word
        if word in item_keywords:
            extracted_item = word
        if word in stat_keywords:
            extracted_stat = word

    return {"role": extracted_role, "champion": extracted_champion, "item": extracted_item, "stat": extracted_stat, "nickname": extracted_nickname, "tag": extracted_tag}

# 유저 질문을 분석하여 API 호출
def handle_user_question(question):
    # 질문에서 역할과 챔피언 추출
    entities = extract_entities(question)
    role = entities['role']
    champion = entities['champion']
    
    if entities["item"]:
        if role and champion:
            # 아이템 정보 API 호출
            item_data = get_item_info(role, champion)
            # 데이터 포맷팅
            formatted_response = format_item_info(item_data, role, champion)
            return formatted_response
        else:
            return "유효한 역할이나 챔피언 정보가 없습니다."

    elif entities["stat"]:
        # 전적 API 호출 (닉네임#태그 형식 확인)
        if entities["nickname"] and entities["tag"]:
            stats = get_user_stats(entities["nickname"], entities["tag"])
            formatted_stats = format_user_stats(stats)
            return formatted_stats
    
    elif "승률" in entities["stat"]:
        if champion:
            # 승률 API 호출
            winrate = get_winrate(champion)
            return f"{champion} 챔피언의 승률: {winrate.get('winrate', '정보 없음')}%"
    
    else:
        return "알 수 없는 질문입니다."

In [17]:
import json

# 자동으로 질문-응답 데이터 저장하는 함수
def save_question_answer(question, answer):
    data = {
        "question": question,
        "answer": answer
    }
    
    # 데이터가 저장될 파일 경로
    file_path = 'qa_dataset.json'
    
    # 기존 데이터 불러오기
    try:
        with open(file_path, 'r') as file:
            qa_data = json.load(file)
    except FileNotFoundError:
        qa_data = []
    
    # 새로운 데이터 추가
    qa_data.append(data)
    
    # JSON 데이터를 UTF-8 인코딩으로 저장
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(qa_data, file, ensure_ascii=False, indent=4)


In [8]:
def auto_create(question):
    response = handle_user_question(question)
    save_question_answer(question, response)

In [18]:
# 테스트 1: "탑 아트록스 아이템 알려줘"
question1 = "탑 아트록스 아이템 알려줘"
response1 = handle_user_question(question1)
save_question_answer(question1, response1)
print(response1)

# 테스트 2: "미드 아리 아이템 알려줘"
question2 = "미드 아리 아이템 알려줘"
response2 = handle_user_question(question2)
print(response2)

챔피언 아트록스 (탑)에 대한 추천 아이템과 룬을 안내해 드리겠습니다.
주 룬: 정복자 (정밀), 승전보 (전설), 전설: 가속 (전설), 최후의 저항 (결단)
부 룬: 뼈 방패 (결의), 소생 (결의)
추천 아이템: 칠흑의 양날 도끼, 선혈포식자, 드락사르의 황혼검, 헤르메스의 발걸음, 예언자의 렌즈
챔피언 아리 (미드)에 대한 추천 아이템과 룬을 안내해 드리겠습니다.
주 룬: 감전 (지배), 피의 맛 (지배), 사냥의 증표 (지배), 궁극의 사냥꾼 (지배)
부 룬: 마나순환 팔찌 (마법), 깨달음 (마법)
추천 아이템: 마법사의 신발, 도란의 반지, 만년서리, 그림자불꽃, 존야의 모래시계, 망원형 개조


In [36]:
question1 = "조선제일GUM#KR 전적 알려줘"
response1 = handle_user_question(question1)
print(response1)  # 전적 정보 반환

전적 정보:

챔피언: Ekko
킬: 8
데스: 9
어시스트: 11
딜량: 10111
피해량: 13320
딜량/분: 677.83
피해량/분: 892.96
딜량/피해량 비율: 0.76
시야: 0
힐: 1169
CS: 13
게임 시간(초): 895

챔피언: Taliyah
킬: 6.5
데스: 5
어시스트: 17.5
딜량: 41171
피해량: 23001
딜량/분: 2141.2
피해량/분: 1182.62
딜량/피해량 비율: 1.84
시야: 0
힐: 790.5
CS: 45
게임 시간(초): 1160

챔피언: Jhin
킬: 6.5
데스: 3
어시스트: 8
딜량: 25386.5
피해량: 15206.5
딜량/분: 1544.1
피해량/분: 935.26
딜량/피해량 비율: 1.66
시야: 0
힐: 3349.5
CS: 43
게임 시간(초): 978

챔피언: Vi
킬: 3
데스: 7
어시스트: 6
딜량: 8462
피해량: 13160
딜량/분: 952.57
피해량/분: 1481.43
딜량/피해량 비율: 0.64
시야: 0
힐: 1684
CS: 15
게임 시간(초): 533

챔피언: Ambessa
킬: 6.5
데스: 5
어시스트: 6
딜량: 24242
피해량: 32389.5
딜량/분: 1588.09
피해량/분: 2124.9
딜량/피해량 비율: 0.75
시야: 0
힐: 7140.5
CS: 27.5
게임 시간(초): 912.5

챔피언: Caitlyn
킬: 8
데스: 7
어시스트: 14
딜량: 17228
피해량: 11722
딜량/분: 1719.93
피해량/분: 1170.25
딜량/피해량 비율: 1.47
시야: 0
힐: 1038
CS: 23
게임 시간(초): 601

챔피언: Ashe
킬: 5
데스: 4.33
어시스트: 12.33
딜량: 39871.33
피해량: 28023.33
딜량/분: 1741.08
피해량/분: 1178.49
딜량/피해량 비율: 1.58
시야: 3
힐: 5824.33
CS: 103.67
게임 시간(초): 1330.67

챔피언: Gwen
킬: 8
데스: 9
어시스트

In [19]:
import json

file_path = 'C:/Users/dlehr/OneDrive/바탕 화면/lol_data/jupyter/qa_dataset.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    print(data)

[{'question': '탑 아트록스 아이템 알려줘', 'answer': '챔피언 아트록스 (탑)에 대한 추천 아이템과 룬을 안내해 드리겠습니다.\n주 룬: 정복자 (정밀), 승전보 (전설), 전설: 가속 (전설), 최후의 저항 (결단)\n부 룬: 뼈 방패 (결의), 소생 (결의)\n추천 아이템: 칠흑의 양날 도끼, 선혈포식자, 드락사르의 황혼검, 헤르메스의 발걸음, 예언자의 렌즈'}]


In [28]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import Dataset

# LLaMA 모델과 토크나이저 로드
model_name = "decapoda-research/llama-7b-hf"  # LLaMA 7B 모델 (또는 다른 모델)
model = LlamaForCausalLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# eos_token을 pad_token으로 설정 (필요한 경우)
tokenizer.pad_token = tokenizer.eos_token  # eos_token을 pad_token으로 설정

# JSON 파일 경로
file_path = './qa_dataset.json'

# 데이터셋으로 변환
dataset = Dataset.from_json(file_path)

# 데이터 전처리 함수
def preprocess_data(examples):
    # 질문과 응답을 하나로 합치고 토큰화 (문자열로 결합)
    input_text = [str(q) + " " + tokenizer.eos_token + " " + str(a) for q, a in zip(examples['question'], examples['answer'])]
    output_text = [str(a) + tokenizer.eos_token for a in examples['answer']]

    # 토큰화
    input_ids = tokenizer(input_text, truncation=True, padding='max_length', max_length=128, return_tensors="pt").input_ids
    labels = tokenizer(output_text, truncation=True, padding='max_length', max_length=128, return_tensors="pt").input_ids

    # 'input_ids'와 'labels'를 각각 리스트 형태로 반환
    return {'input_ids': input_ids.tolist(), 'labels': labels.tolist()}

# 데이터셋 전처리
dataset = dataset.map(preprocess_data, batched=True)


OSError: decapoda-research/llama-7b-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [27]:
from transformers import Trainer, TrainingArguments

# 학습 매개변수 설정
training_args = TrainingArguments(
    output_dir="./output",          # 학습 결과 저장 디렉토리
    evaluation_strategy="epoch",    # 에폭마다 평가
    learning_rate=2e-5,             # 학습률
    per_device_train_batch_size=2,  # 배치 크기
    num_train_epochs=3,            # 에폭 수
    weight_decay=0.01,              # 가중치 감소
)

# Trainer 설정
trainer = Trainer(
    model=model,                  # 학습할 모델
    args=training_args,           # 학습 매개변수
    train_dataset=dataset,        # 학습 데이터셋
)

# 학습 시작
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [ ]:
# 학습된 모델 저장
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

In [ ]:
# fine-tuned 모델과 토크나이저 로드
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

# 모델 예측
def generate_response(question):
    input_ids = fine_tuned_tokenizer.encode(question + fine_tuned_tokenizer.eos_token, return_tensors='pt')
    output = fine_tuned_model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
    return fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)

# 예시 질문에 대한 응답
question = "탑 아트록스 아이템 알려줘"
response = generate_response(question)
print(response)